In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. 载入数据集

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

print('Training set:')
print('       Data :\t shape:', np.shape(x_train), '\t type:', x_train.dtype)
print('       Label:\t shape:', np.shape(y_train), '\t\t type:', y_train.dtype)
print('Testing set :')
print('       Data :\t shape:', np.shape(x_test), '\t type:', x_test.dtype)
print('       Label:\t shape:', np.shape(y_test), '\t\t type:', y_test.dtype)

Training set:
       Data :	 shape: (60000, 28, 28) 	 type: uint8
       Label:	 shape: (60000,) 		 type: uint8
Testing set :
       Data :	 shape: (10000, 28, 28) 	 type: uint8
       Label:	 shape: (10000,) 		 type: uint8


# 2. 数据预处理

In [3]:
# 转化为 float 型并归一化
x_train = x_train.reshape(60000,-1).astype(np.float32)/255
x_test = x_test.reshape(10000,-1).astype(np.float32)/255
print(np.shape(x_train), np.shape(x_test))

def OneHot(y, num_class = 10):
    new_y = tf.one_hot(indices=y, depth = num_class, dtype=tf.float32)
    with tf.Session() as sess:
        sess.run(new_y)
    return new_y
y_train = OneHot(y_train, 10)
y_test = OneHot(y_test, 10)
print(y_train, y_test)

(60000, 784) (10000, 784)
Tensor("one_hot:0", shape=(60000, 10), dtype=float32) Tensor("one_hot_1:0", shape=(10000, 10), dtype=float32)


# 3.设置超参数

In [4]:
BATCH_SIZE = 100
NUM_BATCH = len(x_train)//BATCH_SIZE
NUM_EPOCH = 20
LEARNING_RATE = 0.2

# 4. 构建网络

In [5]:
# 4.1. 创建一个简单的神经网络
class Model(object):
    def __init__(self):
        self.W = tf.Variable(tf.zeros([784, 10]))
        self.b = tf.Variable(tf.zeros([10]))

    def __call__(self, x):
        return tf.nn.softmax(tf.matmul(x, self.W) + self.b)

# 4.2 Loss Function
def Loss(predicted_y_prob, y):
    return tf.reduce_mean(tf.square(y - predicted_y_prob))

# 4.3 Accuracy Evaluator
def Accuracy(predicted_y_prob, desired_y):
    correct_prediction = tf.equal(tf.argmax(desired_y,1), tf.argmax(predicted_y_prob,1))#argmax返回一维张量中最大的值所在的位置
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# 4.4 使用TensoFlow创建可用于训练的数据集
TrainDataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
TrainDataset = TrainDataset.shuffle(buffer_size=5000)
TrainDataset = TrainDataset.batch(100)
TrainDataset = TrainDataset.repeat()
train_iterator = TrainDataset.make_one_shot_iterator()

TestDataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
TestDataset = TestDataset.batch(len(x_test))
TestDataset = TestDataset.repeat()
test_iterator = TestDataset.make_one_shot_iterator()

# 4.5 Optimizer
model = Model()

x, y = train_iterator.get_next()
loss = Loss(model(x), y)
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)

x_test, y_test = test_iterator.get_next()
accuracy = Accuracy(model(x_test), y_test)

# 5. 训练、评估网络

In [6]:
#初始化变量
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(NUM_EPOCH):
        for batch in range(NUM_BATCH):      
            sess.run(train_step)
        
        acc = sess.run(accuracy)         
        print("Iter %2d : Testing Accuracy %.4f"%(epoch+1, acc))

Iter  1 : Testing Accuracy 0.8389
Iter  2 : Testing Accuracy 0.8724
Iter  3 : Testing Accuracy 0.8829
Iter  4 : Testing Accuracy 0.8898
Iter  5 : Testing Accuracy 0.8947
Iter  6 : Testing Accuracy 0.8975
Iter  7 : Testing Accuracy 0.9007
Iter  8 : Testing Accuracy 0.9024
Iter  9 : Testing Accuracy 0.9042
Iter 10 : Testing Accuracy 0.9054
Iter 11 : Testing Accuracy 0.9059
Iter 12 : Testing Accuracy 0.9079
Iter 13 : Testing Accuracy 0.9092
Iter 14 : Testing Accuracy 0.9103
Iter 15 : Testing Accuracy 0.9106
Iter 16 : Testing Accuracy 0.9105
Iter 17 : Testing Accuracy 0.9117
Iter 18 : Testing Accuracy 0.9130
Iter 19 : Testing Accuracy 0.9137
Iter 20 : Testing Accuracy 0.9146
